## 1. 数据进行预处理
将数据读入内存, 并封装在Article类中.

In [50]:
class Article:
    def __init__(self, str):
        self.authors = []
        lines = str.split('\n')
        for line in lines:
            if line is '' or line is None or line == "":
                continue
            key, value = line.split('\t', 1)
            if key == 'author':
                self.authors.append(value)
            elif key == 'title':
                self.title = value
            elif key == 'Conference':
                self.conference = value
    
    def display(self):
        str = ' '.join(self.authors) + ':' + self.title + ',' + self.conference
        print(str)
    
    
def parse_article(filename):
    fin = open(filename)
    content = fin.read()
    blocks = content.split('#########')
    articles = []
    for block in blocks:
        if block is '' or block is None or block == "":
            continue
        art = Article(block)
        # art.display()
        articles.append(art)
    return articles
        
articles = parse_article('data/FilteredDBLP.txt')

## 2. 活跃作者挖掘
根据时间信息，看看哪些人依然活跃，哪些 人不再活跃。

## 3. 频繁项挖掘
经常性在一起合作的 学者，将之称为‘团队’, 根据研究者合作发表论文次数为根据 进行频繁模式挖掘，找出三个人以上的‘团队’.

- 找出支持度support>=5的所有学者

In [74]:
from pymining import itemmining
# transactions = (('a', 'b', 'c'), ('b'), ('a'), ('a', 'c', 'd'), ('b', 'c'), ('b', 'c'))
# relim_input = itemmining.get_relim_input(transactions)
# report = itemmining.relim(relim_input, min_support=2)

authors = [article.authors for article in articles]
relim_input = itemmining.get_relim_input(authors)
report = itemmining.relim(relim_input, min_support=5)

- 找出合作者个数大于3的所有合作团队

In [96]:
counter = 0
for r in report:
    if len(r) >= 3:
        counter += 1
       # print(r)
print(counter)

268


## 4. 主题挖掘
先定出主题词， 然后根据每个‘团队’发表的论文的情况，提炼出这个团队最常涉猎 的主题。

- 根据Latent Dirichlet allocation(LDA)主题模型训练出所有的模型:

In [87]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# compile sample documents into a list
titles = [article.title for article in articles]
doc_set = []
for title in titles:
    doc_set.append(title)

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)

- 使用***作者<--->标题<--->主题***这个映射将作者经常所涉猎的主题进行关联:

In [105]:
topics = ldamodel.get_topics()
# print(ldamodel.show_topic(19, topn=3))
doc2topics = ldamodel.get_document_topics(corpus)
# print(len(doc2topics))
# print(doc2topics[0])
# print(doc2topics[1])
# len(doc2topics)

for idx, article in enumerate(articles[:10]):
    max_pro_topic = 0
    max_pro = 0.0
    # Find the topic_id with maximum probability
    for topic_id, pro in doc2topics[idx]:
        if pro > max_pro:
            max_pro = pro
            max_pro_topic = topic_id
    # print(max_pro_topic)
    topic_terms = [topic_pro[0] for topic_pro in ldamodel.show_topic(max_pro_topic, topn=3)]
    str = ', '.join(article.authors) + "--->" + ', '.join(topic_terms)
    print(str)

Miroslaw Truszczynski, Stefan Woltran--->logic, program, probabilist
Guibing Guo, Jie Zhang, Neil Yorke-Smith--->inform, improv, retriev
Xinwang Liu, Yong Dou, Jianping Yin, Lei Wang 0001, En Zhu--->graph, match, cluster
Mickey Brautbar, Michael J. Kearns, Umar Syed--->factor, matrix, decis
Shirin Sohrabi, Anton V. Riabov, Octavian Udrea--->effici, high, search
Maximilian Nickel, Lorenzo Rosasco, Tomaso A. Poggio--->graph, match, cluster
Kyle Lund, Sam Dietrich, Scott Chow, James C. Boerkoel--->search, tempor, visual
Leandro Soriano Marcolino, Boian Kolev, Samori Price, Sreerag Palangat Veetil, David Jason Gerber, Josef Musil, Milind Tambe--->optim, larg, scale
Russell Bent, Alan Berscheid, G. Loren Toole--->social, person, joint
Vibhav Gogate, Rina Dechter--->robust, estim, hierarch


## 5. 团队和主题多是会随着时间而动态变化
根据自己所定的时 间段(五年，三年，两年或是一年)描述团队的构成状况以及其 研究主题的变化情况。